In [3]:
pip install ms2deepscore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.0/205.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 107.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uni

In [2]:
import requests
import os
from tqdm import tqdm

def download_file(link, file_name):
    response = requests.get(link, stream=True)
    if os.path.exists(file_name):
        print(f"The file {file_name} already exists, the file won't be downloaded")
        return
    total_size = int(response.headers.get('content-length', 0))

    with open(file_name, "wb") as f, tqdm(desc="Downloading file", total=total_size, unit='B', unit_scale=True, unit_divisor=1024,) as bar:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
                bar.update(len(chunk))

model_file_name = "ms2deepscore_model.pt"
spectrum_file_name = "pesticides.mgf"
download_file("https://zenodo.org/records/14290920/files/settings.json?download=1", "ms2deepscore_settings.json")
download_file("https://zenodo.org/records/14290920/files/ms2deepscore_model.pt?download=1", model_file_name)
download_file("https://raw.githubusercontent.com/matchms/ms2deepscore/refs/heads/main/tests/resources/pesticides_processed.mgf", spectrum_file_name)

The file ms2deepscore_settings.json already exists, the file won't be downloaded
The file ms2deepscore_model.pt already exists, the file won't be downloaded
The file pesticides.mgf already exists, the file won't be downloaded


In [3]:
# Creating the Model
from ms2deepscore.models import load_model
model = load_model(model_file_name, allow_legacy=True)

/usr/local/lib/python3.12/dist-packages/ms2deepscore/models/load_model.py:151: RuntimeWarning: Using UNSAFE legacy loading (weights_only=False). Only do this for trusted files.
  warnings.warn(


In [4]:
# Calculating MS2Deepscore scores
from matchms.Pipeline import Pipeline, create_workflow
from matchms.filtering.default_pipelines import DEFAULT_FILTERS
from ms2deepscore import MS2DeepScore

pipeline = Pipeline(create_workflow(query_filters=DEFAULT_FILTERS,
                                    score_computations=[[MS2DeepScore, {"model": model}]]))
report = pipeline.run(spectrum_file_name,
                     # reference_files="path_to_spectrum_file.mgf" ## Add a file here if you want to get scores against a reference library.
                     )
similarity_matrix = pipeline.scores.to_array()

2025-12-07 16:38:42,690:WARNING:matchms:Pipeline:The order of the filters has been changed compared to the Yaml file.


2025-12-07 16:38:42,693:WARNING:matchms:Pipeline:The order of the filters has been changed compared to the Yaml file.


2025-12-07 16:38:42,698:WARNING:matchms:Pipeline:No logging file was defined.Logging messages will not be written to file.


Processing spectra: 0it [00:00, ?it/s]WARNING:pickydict:Key 'name' will be interpreted as 'compound_name'. But this entry already exists. Please use 'compound_name' if you want to replace the entry.
Processing spectra: 7it [00:00, 68.41it/s]WARNING:pickydict:Key 'name' will be interpreted as 'compound_name'. But this entry already exists. Please use 'compound_name' if you want to replace the entry.
Processing spectra: 15it [00:00, 73.31it/s]WARNING:pickydict:Key 'name' will be interpreted as 'compound_name'. But this entry already exists. Please use 'compound_name' if you want to replace the entry.
Processing spectra: 23it [00:00, 72.28it/s]WARNING:pickydict:Key 'name' will be interpreted as 'compound_name'. But this entry already exists. Please use 'compound_name' if you want to replace the entry.
Processing spectra: 31it [00:00, 69.88it/s]WARNING:pickydict:Key 'name' will be interpreted as 'compound_name'. But this entry already exists. Please use 'compound_name' if you want to repla

In [5]:
print(report)

----- Spectrum Processing Report -----
Number of spectra processed: 76
Number of spectra removed: 0
Changes during processing:
                              removed spectra  changed metadata  changed mass spectrum
filter                                                                                
add_parent_mass                             0                76                      0
add_retention_index                         0                76                      0
add_retention_time                          0                76                      0
clean_adduct                                0                76                      0
derive_formula_from_smiles                  0                76                      0
make_charge_int                             0                 0                      0
add_compound_name                           0                 0                      0
derive_adduct_from_name                     0                 0                      0
der

In [6]:
print(similarity_matrix.shape)

(76, 76)


In [7]:
print(similarity_matrix)

[[1.         0.39567415 0.31721568 ... 0.16670259 0.19357387 0.15074877]
 [0.39567415 1.         0.3567378  ... 0.16240767 0.21237039 0.13035795]
 [0.31721568 0.3567378  1.         ... 0.38725827 0.3921025  0.56782875]
 ...
 [0.16670259 0.16240767 0.38725827 ... 1.         0.99078505 0.47407308]
 [0.19357387 0.21237039 0.3921025  ... 0.99078505 1.         0.45381763]
 [0.15074877 0.13035795 0.56782875 ... 0.47407308 0.45381763 1.        ]]


In [8]:
# Calculate embeddings
cleaned_spectra = pipeline.spectra_queries
ms2ds_model = MS2DeepScore(model)
ms2ds_embeddings = ms2ds_model.get_embedding_array(cleaned_spectra)

Computing spectral embeddings ...: 100%|██████████| 76/76 [00:02<00:00, 28.54it/s]


In [9]:
ms2ds_embeddings

array([[-0.86432904, -0.99871385,  0.23028606, ..., -0.20689027,
         0.8267501 , -0.69716865],
       [-0.45379511, -0.94319844, -0.2845467 , ..., -0.02428324,
        -0.59131169,  0.42027235],
       [-0.8610484 , -0.32745886,  0.39620909, ...,  0.62359434,
         0.81631714, -0.36466593],
       ...,
       [-0.88536602,  0.93186814,  0.06120966, ..., -0.45469359,
         0.0580497 ,  0.40203047],
       [-0.87119734,  0.95781434,  0.18429664, ..., -0.43923515,
        -0.13216279,  0.28982326],
       [-0.44632268, -0.09382542,  0.75702566, ..., -0.0366261 ,
         0.07153989, -0.1326734 ]], shape=(76, 500))

In [10]:
# Save Embeddings
import numpy as np
np.save("ms2ds_embeddings.npy", ms2ds_embeddings)

In [11]:
# Download Embeddings
from google.colab import files
files.download("ms2ds_embeddings.npy")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
smiles = []
for spectrum in cleaned_spectra:
  smiles.append(spectrum.get("smiles"))
print(len(smiles))

76


In [23]:
import pandas as pd
from google.colab import files
pd.DataFrame({"smiles": smiles}).to_csv("smiles.csv", index=False)
files.download("smiles.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>